### Creating manifest file

> Run only once!!!



In [ ]:
# Connect to drive to access files only for colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Manifest file is created in csv format
import os
import csv
import pandas as pd
import numpy as np

# Define the paths
audio_folder = "/content/drive/Shareddrives/Priya_speechlab/wav2vec2/FSI/test_audio"               # path to audio
transcription_folder = "/content/drive/Shareddrives/Priya_speechlab/wav2vec2/FSI/test_prompts"    # path to transcriptions
output_csv = "/content/drive/Shareddrives/Priya_speechlab/wav2vec2/manifest_files/FSI_test.csv"  # path to store manifest file

# Open the CSV file for writing


# with open(output_csv, mode="w", newline="") as file:
#     writer = csv.writer(file)
#     writer.writerow(["path", "transcription"])  # CSV header

#     # Loop through each audio file
#     for audio_file in os.listdir(audio_folder):
#         if audio_file.endswith(".wav"):
#             # Get corresponding transcription file
#             audio_base = os.path.splitext(audio_file)[0]  # Get the base name (audio1, audio2, etc.)
#             transcription_file = os.path.join(transcription_folder, f"{audio_base}.txt")

#             # Read the transcription text
#             if os.path.exists(transcription_file):
#                 with open(transcription_file, "r") as f:
#                     transcription = f.read().strip()

#                 # Write the audio path and transcription to the CSV
#                 writer.writerow([os.path.join(audio_folder, audio_file), transcription])
#             else:
#                 print(f"Transcription for {audio_file} not found!")
# print("Manifest file created successfully!")

In [ ]:
# View the manifest file
manifest=pd.read_csv(output_csv)
manifest['transcription'].head(15)

,transcription
0,ஜலம்
1,கத்தியைத் தீட்டாமே புத்தியைத் தீட்டு
2,மயில் அகவும்
3,குரங்கு பந்தை எறிந்தது
4,பட பட
5,புஷ்பம்
6,யானை பிளிறும்
7,செய்வன திருந்த செய்
8,தராதரம்
9,மனம் திறந்து பேசு


### Install and import necessary libraries

In [ ]:
# !pip install torch torchaudio transformers
# !pip install soundfile
# !pip install librosa
!pip install datasets

In [ ]:
!pip install datasets pyarrow==8.0.0 # install compatible versions of datasets and pyarrow

INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarro

In [ ]:
import torch
import torchaudio
import soundfile as sf
import librosa

### Preprocess audio and tokenize transcriptions

In [ ]:
# Load manifest file for training
from datasets import load_dataset

train_manifest = "/content/drive/Shareddrives/Priya_speechlab/wav2vec2/manifest_files/FSI_train.csv" # path to train data
# test_manifest = "/content/drive/Shareddrives/Priya_speechlab/wav2vec2/manifest_files/FSI_test.csv"  # path to test data

train_dataset = load_dataset("csv", data_files={"train":train_manifest}, split="train")
# test_dataset = load_dataset("csv", data_files={"test":test_manifest})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(train_dataset)
#print(test_dataset)

Dataset({
    features: ['path', 'transcription'],
    num_rows: 292
})


In [ ]:
# Load model and processor
# Amrrs/wav2vec2-large-xlsr-53-tamil
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")
model = Wav2Vec2ForCTC.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at Amrrs/wav2vec2-large-xlsr-53-tamil were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Amrrs/wav2vec2-large-xlsr-53-tamil and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRA

In [ ]:
# Preprocessing function to load and tokenize data
def preprocess_data(batch):
    # Load the audio file
    speech_array, sampling_rate = torchaudio.load(batch['path'])
    batch['speech'] = speech_array.numpy()

    # Process the audio: Extract input values from speech
    batch["input_values"] = processor(batch["speech"], sampling_rate=sampling_rate).input_values[0]

    # Tokenize the transcription
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids

    return batch

# Apply the preprocessing function to the dataset
dataset = train_dataset.map(preprocess_data, remove_columns=train_dataset.column_names)
# test_dataset = test_dataset.map(preprocess_data, remove_columns=train_dataset.column_names)

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

### Training

In [ ]:
from transformers import TrainingArguments, Trainer

# Define training args
training_args = TrainingArguments(
    output_dir="/content/drive/Shareddrives/Priya_speechlab/wav2vec2/wav2vec2-tamil-FSI-mild",   # Path to save model and checkpoints
    evaluation_strategy="steps",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    warmup_steps=500,
    save_total_limit=2,
    #remove_unused_columns=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
    #eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)


In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss


TrainOutput(global_step=27, training_loss=3.3401336669921875, metrics={'train_runtime': 200.5916, 'train_samples_per_second': 4.367, 'train_steps_per_second': 0.135, 'total_flos': 1.105906923583608e+17, 'train_loss': 3.3401336669921875, 'epoch': 2.8421052631578947})

In [ ]:
# Save the fine-tuned model and processor
model.save_pretrained("/content/drive/Shareddrives/Priya_speechlab/wav2vec2/wav2vec2-tamil-FSI-mild")
processor.save_pretrained("/content/drive/Shareddrives/Priya_speechlab/wav2vec2/wav2vec2-tamil-FSI-mild")


NameError: name 'model' is not defined

### Testing and validation

In [ ]:
# load the model and processor
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/Shareddrives/Priya_speechlab/wav2vec2/wav2vec2-tamil-FSI-mild/checkpoint-27")

In [ ]:
import torchaudio
import soundfile as sf
import librosa

# Load the audio file
speech_array, sampling_rate = librosa.load("/content/drive/Shareddrives/Priya_speechlab/wav2vec2/FSI/test_audio/FSI293.wav", sr=16000) # Load audio and resample to 16000Hz

# Preprocess the audio (resample and extract features)
input_values = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt").input_values

# Use the model to predict the transcription (Inference)
with torch.no_grad():
    logits = model(input_values).logits

# Get the predicted tokens (IDs) by taking the argmax across logits
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the predicted IDs back into text (transcription)
transcription = processor.batch_decode(predicted_ids)[0]

print("Transcription:", transcription)

Transcription: குறுவே சித்தியர் வலைங்கிமார்


In [ ]:
# Load the audio file
speech_array, sampling_rate = librosa.load("/content/drive/Shareddrives/Priya_speechlab/wav2vec2/FSI/test_audio/FSI294.wav", sr=16000) # Load audio and resample to 16000Hz

# Preprocess the audio (resample and extract features)
input_values = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt").input_values

# Use the model to predict the transcription (Inference)
with torch.no_grad():
    logits = model(input_values).logits

# Get the predicted tokens (IDs) by taking the argmax across logits
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the predicted IDs back into text (transcription)
transcription = processor.batch_decode(predicted_ids)[0]

print("Transcription:", transcription)

Transcription: தீக்கின எற்பாடு திய்


In [ ]:
import torchaudio
import soundfile as sf
import librosa

# Load the audio file
speech_array, sampling_rate = librosa.load("/content/drive/Shareddrives/Priya_speechlab/wav2vec2/FSI/test_audio/FSI365.wav", sr=16000) # Load audio and resample to 16000Hz

# Preprocess the audio (resample and extract features)
input_values = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt").input_values

# Use the model to predict the transcription (Inference)
with torch.no_grad():
    logits = model(input_values).logits

# Get the predicted tokens (IDs) by taking the argmax across logits
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the predicted IDs back into text (transcription)
transcription = processor.batch_decode(predicted_ids)[0]

print("Transcription:", transcription)

Transcription: வினியாள வள்ஞ்சு நாட்டுவ லச்றி


### Result of base model

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")
model = Wav2Vec2ForCTC.from_pretrained("Amrrs/wav2vec2-large-xlsr-53-tamil")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at Amrrs/wav2vec2-large-xlsr-53-tamil were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Amrrs/wav2vec2-large-xlsr-53-tamil and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRA

In [ ]:
import torchaudio
import soundfile as sf
import librosa
import torch

# Load the audio file
speech_array, sampling_rate = librosa.load("/content/drive/Shareddrives/Priya_speechlab/wav2vec2/FSI/test_audio/FSI294.wav", sr=16000) # Load audio and resample to 16000Hz

# Preprocess the audio (resample and extract features)
input_values = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt").input_values

# Use the model to predict the transcription (Inference)
with torch.no_grad():
    logits = model(input_values).logits

# Get the predicted tokens (IDs) by taking the argmax across logits
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the predicted IDs back into text (transcription)
transcription = processor.batch_decode(predicted_ids)[0]

print("Transcription:", transcription)

Transcription: தீக்கின எற்வாடு திய்
